In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import time

In [2]:
# Chrome Browser와 Chrome Driver Version 확인
chrome_options = webdriver.ChromeOptions()

# 요청
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
url = 'https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do'
driver.get(url)

time.sleep(5)

# 지점 선택 박스
region_box = Select(driver.find_element(By.ID, "select-stn"))
region_box.select_by_visible_text('서울(유)')

# 각 월에 대해 데이터 가져오기
all_dates = []
all_weather = []

# 현재 연도와 월 가져오기
current_year = datetime.now().year
current_month = datetime.now().month

# 데이터를 가져올 범위 설정 (4월부터 10월까지, 2023년부터 현재 연도까지)
for year in range(2022, current_year + 1):  # 연도 변경
    end_month = 10 if year < current_year else current_month  # 현재 연도인 경우 현재 월까지만 가져오자
    for month in range(4, end_month + 1):  # 4월부터 현재 6월까지
        # 월별 선택 박스
        month_box = Select(driver.find_element(By.ID, "select-mm"))
        month_box.select_by_visible_text(f'{month}월')
        time.sleep(5)

        # 연도 선택 박스
        year_box = Select(driver.find_element(By.ID, "select-yy"))
        year_box.select_by_visible_text(f'{year}년')
        time.sleep(5)

        # 요소 선택 박스
        element_box = Select(driver.find_element(By.ID, "select-obs"))
        element_box.select_by_visible_text('날씨')
        time.sleep(5)

        # 선택 버튼 클릭
        select_button = driver.find_element(By.CSS_SELECTOR, "input[type='submit'][data-form-role='submit'][data-form-id='default-form']")
        select_button.click()
        time.sleep(5)

        # 데이터 테이블 로딩 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".table-col.table-cal")))
        
        # 데이터 테이블 가져오기
        data_table = driver.find_element(By.CSS_SELECTOR, ".table-col.table-cal")
        # print("Data table successfully located for", year, "년", month, "월")
        
        # 각 날짜와 날씨 정보 파싱
        rows = data_table.find_elements(By.XPATH, ".//tbody/tr")
        
        # 날짜와 날씨 정보를 번갈아 가며 추출
        dates = []
        weather = []
        
        for i in range(0, len(rows), 2):
            date_elements = rows[i].find_elements(By.XPATH, ".//td/span")
            weather_elements = rows[i+1].find_elements(By.XPATH, ".//td/span")
            
            for date_element in date_elements:
                date = date_element.text.strip()
                if date:
                    dates.append(date)
            
            for weather_element in weather_elements:
                weather_condition = weather_element.text.strip()
                if not weather_condition:
                    weather_condition = "맑음"  # 날씨 정보가 없을 경우 '맑음'으로 처리
                elif weather_condition in ["박무", "연무", "황사", "햇무리"]:
                    weather_condition = "맑음"
                elif weather_condition in ["안개비", "소나기"]:
                    weather_condition = "비"
                weather.append(weather_condition)
        
        # 날짜와 날씨 정보 출력하여 확인
        # print("Dates for", year, "년", month, "월:")
        # print(dates)
        # print("\nWeather for", year, "년", month, "월:")
        # print(weather)
        
        # 날짜와 날씨 정보 저장
        all_dates.extend([f"{year}-{month:02d}-{date}" for date in dates])
        all_weather.extend(weather)

# 최종 데이터 출력 또는 파일로 저장
for date, weather in zip(all_dates, all_weather):
    print(f"[{date}], [{weather}]")

WebDriverException: Message: Service /Users/sori/.wdm/drivers/chromedriver/mac64/125.0.6422.141/chromedriver-mac-arm64/chromedriver unexpectedly exited. Status code was: -9
